## Installing some of the dependencies 

In [3]:
! pip install psycopg2 
! pip install requests
! pip install SQLAlchemy

  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-macosx_10_15_x86_64.whl size=135420 sha256=5598901e1b955c3b360517648c65aa5c97c0497d1941927807aefe604a03ed50
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2


# Importing dependencies 

In [10]:
import pandas as pd
import numpy
import requests
import json
import csv

# Extraction Layer: Extracting the dataset from RapidAPI

In [17]:

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"2000"}

headers = {
	"x-rapidapi-key": "ce3693686amshd8a81ba62ce435ep1113afjsn137d2b9aa172",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()

# saving the data as a file

filename = 'propertyRecords.json'

# to write the file into the filename
with open (filename, 'w') as file:
    json.dump(data,file, indent = 4)

In [11]:
# to read into a dataframe

propertyrecords_df = pd.read_json('propertyRecords.json')


In [31]:
propertyrecords_df['taxAssessment']

0              {'2022': {'value': 11056, 'land': 11056}}
1      {'2022': {'value': 14053, 'land': 845, 'improv...
2      {'2019': {'value': 329900, 'land': 75000, 'imp...
3      {'2022': {'value': 40000, 'land': 40000}, '202...
4      {'2018': {'value': 60879, 'land': 24560, 'impr...
                             ...                        
495    {'2020': {'value': 200300, 'land': 60000, 'imp...
496    {'2021': {'value': 298150, 'land': 85000, 'imp...
497    {'2023': {'value': 417100, 'land': 203000, 'im...
498    {'2024': {'value': 665800, 'land': 96700, 'imp...
499        {'2023': {'value': 420, 'improvements': 420}}
Name: taxAssessment, Length: 500, dtype: object

In [27]:
# approach of cleaning
propertyrecords_df['features'] = propertyrecords_df['features'].apply(json.dumps)

# second approach to find and replace the nulls
propertyrecords_df.fillna({
        'bedrooms': 0,
        'addressLine2': 'Not available', 
        'squareFootage': 0,
        'yearBuilt': 0,
        'features': 'None',
        'assessorID': 'Unknown',
        'legalDescription': 'Not available',
        'subdivision': 'Not available', 
         'zoning': 'Unknown', 
         'bathrooms': 0, 
         'lotSize': 0,
         'propertyType': 'Unknown', 
         'taxAssessment': 'Not available',
        'propertyTaxes':  'Not available', 
         'lastSalePrice': 0,
        'lastSaleDate': 'Not available',
        'owner': 'Unknown',
        'ownerOccupied': 0,
        'addressLine2': 'Unknown',
        'county': 'Not available'}, inplace =True)

# to extract the id from the address
propertyrecords_df['id'] = propertyrecords_df['id'].apply(lambda x: id(x))
    
    

    



In [28]:
fact_columns = ['id','bedrooms', 'squareFootage','bathrooms', 'lotSize','lastSalePrice','lastSaleDate']
fact_table = propertyrecords_df[fact_columns]

In [29]:
fact_table

,id,bedrooms,squareFootage,bathrooms,lotSize,lastSalePrice,lastSaleDate
0,5008314160,2.0,1254.0,1.0,9800.0,9200.0,2017-03-01T00:00:00.000Z
1,5008308096,3.0,1438.0,2.0,6675.0,0.0,Not available
2,5008313200,3.0,1387.0,1.0,6098.0,330000.0,2018-09-28T00:00:00.000Z
3,5008308496,5.0,2210.0,2.0,9583.0,2264900.0,2023-09-11T00:00:00.000Z
4,5008308896,0.0,3720.0,0.0,15272.0,0.0,Not available
...,...,...,...,...,...,...,...
495,5010957424,3.0,1577.0,3.0,7623.0,229000.0,2020-04-28T00:00:00.000Z
496,5010957824,4.0,2266.0,2.5,8500.0,0.0,1998-08-05T00:00:00.000Z
497,5010958304,3.0,1520.0,2.0,8000.0,455000.0,2021-06-18T00:00:00.000Z
498,5010502640,3.0,1088.0,2.0,0.0,550000.0,2017-11-03T00:00:00.000Z
